## 初始化

In [23]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import sys
sys.path.append('../')

print(sys.path)

['/mnt/workspace/code/ModelScopeGPT/agent/demo', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/tinycudann-1.6-py3.8-linux-x86_64.egg', '../', '../', '../']


## Load cfg and initialize

In [25]:
from dotenv import load_dotenv

load_dotenv('../config/.env', override=True)

import os

from modelscope.utils.config import Config

model_cfg_file = os.getenv('LLM_CONFIG_FILE')
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE') 

print(model_cfg_file)
print(tool_cfg_file)

model_cfg = Config.from_file(model_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)

../config/cfg_model.json
../config/cfg_tool.json


In [26]:
# tool检验

from modelscope_agent.tools import LangchainTool
from langchain.tools import ShellTool, ReadFileTool


shell_tool = LangchainTool(ShellTool())
read_tool = LangchainTool(ReadFileTool())


additional_tool_list = {
    shell_tool.name: shell_tool,
    read_tool.name: read_tool
}

print(shell_tool(commands=["echo 'Hello World!'", "ls"]))
print(additional_tool_list)


{'result': 'Hello World!\n1.txt\ndemo_chatgpt.ipynb\ndemo.ipynb\ndemo_local_llm.ipynb\nnew_directory\ntmp\ntool_agent_finetune\n'}
{'terminal': <modelscope_agent.tools.plugin_tool.LangchainTool object at 0x7fa5b48ed730>, 'read_file': <modelscope_agent.tools.plugin_tool.LangchainTool object at 0x7fa5b48ed910>}


In [27]:
from modelscope_agent.agent import AgentExecutor
from modelscope_agent.llm.openai import OpenAi
from modelscope_agent.prompt import DEFAULT_CHATGPT_PROMPT_TEMPLATE, MSPromptGenerator



prompt_generator = MSPromptGenerator(DEFAULT_CHATGPT_PROMPT_TEMPLATE)
llm = OpenAi(model_cfg)
agent = AgentExecutor(llm, tool_cfg, additional_tool_list=additional_tool_list, prompt_generator=prompt_generator)

available_tool_list = [
    'terminal',
    'read_file'
]

agent.set_available_tools(available_tool_list)

## tool检验

In [28]:
agent.reset()
agent.run('use tool to execute command \'ls\'')

**************************************************round 1**************************************************

<IPython.core.display.JSON object>

Sure, I can help you with that. Please use the following command:

'

|action: terminal, action_args: {'commands': 'ls'}


**************************************************round 2**************************************************

The command 'ls' has been executed and the following files and directories have been found: 1.txt, demo_chatgpt.ipynb, demo.ipynb, demo_local_llm.ipynb, new_directory, tmp, tool_agent_finetune.

|action: None, action_args: None


[{'result': '1.txt\ndemo_chatgpt.ipynb\ndemo.ipynb\ndemo_local_llm.ipynb\nnew_directory\ntmp\ntool_agent_finetune\n'}]

In [29]:
agent.reset()
agent.run('please help read the file content of 1.txt')

**************************************************round 1**************************************************

<IPython.core.display.JSON object>

Sure, I can help you with that. To read the file content of 1.txt, we can use the "read_file" tool. Here is the command: 

'

|action: read_file, action_args: {'file_path': '1.txt'}


**************************************************round 2**************************************************

The content of the 1.txt file is "hello!".

|action: None, action_args: None


[{'result': 'hello!'}]

In [36]:
from modelscope_agent.tools import HFTool


hf_text_translation_tool = HFTool()

additional_tool_list = {
    'hf_text_translation': hf_text_translation_tool
}

agent = AgentExecutor(llm, tool_cfg, additional_tool_list=additional_tool_list, prompt_generator=prompt_generator)
agent.reset()

available_tool_list = [
    'hf_text_translation'
]
agent.set_available_tools(available_tool_list)
agent.run("translate the following English into French: 'hello' ")

**************************************************round 1**************************************************

<IPython.core.display.JSON object>

Sure, I can help you with that. 



|action: hf_text_translation, action_args: {'text': 'hello', 'src_lang': 'English', 'tgt_lang': 'French'}


AttributeError: module 'torch.backends' has no attribute 'mps'